In [22]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# Bibliothèques de fonctions STEAN
from stean_plus import instanceST 

# Fichier de paramètres
import configparser # Permet de parser le fichier de paramètres

# Lecture du fichier de configuration
config = configparser.RawConfigParser() # On créé un nouvel objet "config"
config.read('param.conf') # On lit le fichier de paramètres

# Déclaration des variables ########################################################################### 
# chemin du fichier des données historiques à importer
path_histo_file = config['HISTORY']['histo_file']

# Nom du datastream cible
#datastream_name ='nom_datastream'
datastream_name = config['HISTORY']['datastream_name']

# Création d'une instance SensorThing ###########################################################################
#sessionST=instanceST('url de votre instance STEAN', 'login de votre instance STEAN', 'mot de passe de votre instance STEAN')
url_STEAN = config['SERVICE']['url']
login = config['SERVICE']['user']
password = config['SERVICE']['pw']
sessionST=instanceST(url_STEAN, login, password)

sessionST.connexion() # connexion au serveur

# Avant d'effectuer l'importation du fichier CVS, il est nécessaire de préparer un json identifiant le Datastream et le FOI associé

# Demande d'information sur un Datastream selon son nom
print("********************\nInformations du datastream cible -> ",datastream_name)

options="expand=FeatureOfInterest&$select=id&$filter=name eq '" + datastream_name + "'"
info_datastream = sessionST.getInfo('Datastreams',options)
id_datastream = str(info_datastream[0]['@iot.id'])
id_FOI = str(info_datastream[0]['FeatureOfInterest']['@iot.id'])
print('Identifiantt du datastream: ',id_datastream)
print('Identfiant de son FOI: ', id_FOI)

# Datastream à cibler - création d'un json à joindre à la requête d'importation
print("\n********************\nCréation du json du datastream cible",datastream_name)
data_json =  {"header": "False", "nan": "True", "columns": { "1": { "Datastream":  id_datastream, "FeaturesOfInterest": id_FOI }}}
print(data_json)

# Envoi du fichier de données
print('\n********************\nEnvoi du fichier CSV ->', path_histo_file)
print('Patientez ...')
response = sessionST.postCsvFile(path_histo_file, data_json)
if response.status_code == 201:
    print("Post OK", response.text)
if response.status_code!= 201:
    print("error: ", response.text)

sessionST.log_out()

# Affcihage des données importées au format graph
print('\n********************\nCliquez sur le lien suivant:\n')
requete =url_STEAN + "Datastreams(" + id_datastream + ")/Observations?$resultFormat=graph"
print(requete)

Connexion agrhys  à  http://localhost:8029/agrhys/v1.1/
Connexion OK à  http://localhost:8029/agrhys/v1.1/ 

********************
Informations du datastream cible ->  Outlet E30_OTT PLS C_stream level
Identifiantt du datastream:  2
Identfiant de son FOI:  2

********************
Création du json du datastream cible Outlet E30_OTT PLS C_stream level
{'header': 'False', 'nan': 'True', 'columns': {'1': {'Datastream': '2', 'FeaturesOfInterest': '2'}}}

********************
Envoi du fichier CSV -> C:\Users\cgeneste\INRAE\projets\SensorThing\data\histo\E30_niveau_eau_X102_B.csv
Patientez ...
Post OK [
  "Add 376760 on 376800 lines from E30_niveau_eau_X102_B.csv"
]
Déconnexion de  http://localhost:8029/agrhys/v1.1/
Déconnexion OK de  http://localhost:8029/agrhys/v1.1/ 


********************
Cliquez sur le lien suivant:

http://localhost:8029/agrhys/v1.1/Datastreams(2)/Observations?$resultFormat=graph
